# Tool Basics

In [2]:
import rich
from llama_stack_client import Agent, LlamaStackClient, AgentEventLogger
from llama_stack_client.lib.agents.client_tool import client_tool

In [3]:
@client_tool
def get_location(query: str = "location"):
    """
    Provide the location upon request.

    :param query: The query from user
    :returns: Information about user location
    """
    import geocoder
    
    try:
        g = geocoder.ip('me')
        if g.ok:
            return f"Your current location is: {g.city}, {g.state}, {g.country}" # can be modified to return latitude and longitude if needed
        else:
            return "Unable to determine your location"
    except Exception as e:
        return f"Error getting location: {str(e)}"

In [4]:
client = LlamaStackClient(
    base_url="http://localhost:8321",
)

In [5]:
client.models.list()

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/models "HTTP/1.1 200 OK"


[Model(identifier='all-MiniLM-L6-v2', metadata={'embedding_dimension': 384.0}, api_model_type='embedding', provider_id='ollama', type='model', provider_resource_id='all-minilm:latest', model_type='embedding'),
 Model(identifier='llama3.2:3b-instruct-fp16', metadata={}, api_model_type='llm', provider_id='ollama', type='model', provider_resource_id='llama3.2:3b-instruct-fp16', model_type='llm')]

In [6]:
client.toolgroups.list()

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/toolgroups "HTTP/1.1 200 OK"


[ToolGroup(identifier='builtin::rag', provider_id='rag-runtime', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::rag'),
 ToolGroup(identifier='builtin::websearch', provider_id='tavily-search', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::websearch'),
 ToolGroup(identifier='builtin::wolfram_alpha', provider_id='wolfram-alpha', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::wolfram_alpha'),
 ToolGroup(identifier='mcp::filesystem', provider_id='model-context-protocol', type='tool_group', args=None, mcp_endpoint=McpEndpoint(uri='http://localhost:8000/sse'), provider_resource_id='mcp::filesystem')]

In [7]:
client.tools.list()

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/tools in 0.405607 seconds
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/tools in 0.980687 seconds
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools "HTTP/1.1 500 Internal Server Error"


InternalServerError: Error code: 500 - {'detail': 'Internal server error: An unexpected error occurred.'}

In [8]:
agent = Agent(
    client, 
    model="meta-llama/Llama-3.2-3B-Instruct",
    instructions="You are a helpful assistant.",
    tools=[get_location],
)


INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"


In [9]:
user_prompts = [
    "Where am I?",
]
session_id = agent.create_session("test-session")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")

    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
        stream=True,
    )
    for log in AgentEventLogger().log(response):
        log.print()

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/36289912-a0f0-41d6-8bcb-c0adcaa04aef/session "HTTP/1.1 200 OK"


User> Where am I?

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/36289912-a0f0-41d6-8bcb-c0adcaa04aef/session/9d2491f3-f252-48d7-9380-f3b4ecd9bc0f/turn "HTTP/1.1 200 OK"


inference> 400: Invalid value: Model 'meta-llama/Llama-3.2-3B-Instruct' not found
